In [1]:
from pymongo import MongoClient
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import numpy as np
load_dotenv()

True

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
companies = db.get_collection("companies")

### Selección de algunas compañías:
- misma actividad que la propuesta y que dispongamos de su localización exacta
- solo mostraremos el nombre, su actividad, el dinero recaudado y la localización de sus oficinas.

In [3]:
proyec = {"name":1,"category_code": 1, "_id": 0,"total_money_raised":1,
          "offices.city":1, "offices.latitude":1, "offices.longitude":1}
cond = {"category_code": "games_video", "total_money_raised":{"$ne":"$0"},
        "offices.latitude":{"$type": "double"},"offices.longitude":{"$type": "double"},
       }
list(companies.find(cond,proyec))[0]

{'name': 'Lala',
 'category_code': 'games_video',
 'total_money_raised': '$44.2M',
 'offices': [{'city': 'Palo Alto',
   'latitude': 37.451151,
   'longitude': -122.154369}]}

In [4]:
selec = pd.DataFrame(list(companies.find(cond,proyec)))
selec.sample(2)

,name,category_code,total_money_raised,offices
89,2threads,games_video,$300k,"[{'city': 'sydney', 'latitude': -33.884685, 'l..."
75,Outspark,games_video,$23.3M,"[{'city': 'San Francisco', 'latitude': 37.7751..."


In [5]:
selec.total_money_raised = selec.total_money_raised.str.strip()

In [6]:
#creamos nuevas columnas en función del dinero recaudado, su moneda y el total 
selec["amount"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[1]
selec["mil"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[2]
selec["currency"] = selec.total_money_raised.str.extract(r"(.)(\d+|\d+.\d+)(.$)")[0]

In [7]:
selec.currency.unique()

array(['$', '€', '£'], dtype=object)

In [8]:
selec.mil.unique()

array(['M', 'k'], dtype=object)

- como solo hay tres tipos de monedas entre nuestra selección de compañía, elegimos una ciudad por cada moneda
- también que halla ganados millones y no miles.

In [9]:
dol = selec[(selec["currency"] == '$') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
city1 = dol["offices"].values[0][0]["city"]

In [10]:
eur = selec[(selec["currency"] == '€') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
city2 = eur["offices"].values[0][0]["city"]

In [11]:
pounds = selec[(selec["currency"] == '£') & (selec["mil"] == "M")].sort_values("amount",ascending = False).head(1)
city3 = pounds["offices"].values[0][0]["city"]

In [12]:
cities= [city1,city2,city3]

## Ciudades elegidas

In [13]:
cities

['Austin', 'Hamburg', 'Newcastle Upon Tyne']